In [1]:
import requests
import csv
import io
import sqlite3

In [10]:
import requests
import csv
import io
import sqlite3
import datetime
import re
import random

# Create a connection to the SQLite database
db_conn = sqlite3.connect('debt_collection.db')
cursor = db_conn.cursor()

# Create the borrowers table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS borrowers (
        id INTEGER PRIMARY KEY,
        name VARCHAR(50),
        date_of_birth DATE,
        gender VARCHAR(10),
        marital_status VARCHAR(20),
        phone_number VARCHAR(20),
        email_address VARCHAR(50),
        mailing_address VARCHAR(100),
        language_preference VARCHAR(20),
        geographical_location VARCHAR(50),
        credit_score INTEGER,
        loan_type VARCHAR(20),
        loan_amount DECIMAL(10, 2),
        loan_term INTEGER,
        interest_rate DECIMAL(5, 2),
        loan_purpose VARCHAR(50),
        emi DECIMAL(10, 2),
        ip_address VARCHAR(20),
        geolocation VARCHAR(50),
        days_left_to_pay_current_emi INTEGER,
        delayed_payment VARCHAR(10),
        income_level VARCHAR(10),
        financial_status VARCHAR(15)
    )
''')

# Create the repayment_history table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS repayment_history (
        id INTEGER PRIMARY KEY,
        borrower_id INTEGER,
        payment_date DATE,
        payment_mode VARCHAR(20),
        FOREIGN KEY (borrower_id) REFERENCES borrowers (id)
    )
''')

# Insert data into the borrowers table
with open('10k_borrowers_data.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        if len(row) < 21:  # Check if the row has at least 21 columns
            print(f"Skipping row with insufficient columns: {row}")
            continue

        # Convert date of birth to datetime object
        dob = datetime.datetime.strptime(row[1], '%d-%m-%Y').date()

        # Standardize phone number
        phone_number = row[4]
        phone_number = re.sub(r'\D', '', phone_number)  # Remove non-digit characters
        if len(phone_number) < 10:
            phone_number = phone_number.zfill(10)  # Pad with zeros to make 10 digits
        phone_number = '+91' + phone_number  # Add country code

        # Randomly allocate income level and financial status
        income_levels = ['low', 'medium', 'high']
        financial_statuses = ['stable', 'semi-stable', 'unstable']
        income_level = random.choice(income_levels)
        financial_status = random.choice(financial_statuses)

        # Insert data into the borrowers table
        cursor.execute('''
            INSERT INTO borrowers (
                name, date_of_birth, gender, marital_status, phone_number, email_address,
                mailing_address, language_preference, geographical_location, credit_score,
                loan_type, loan_amount, loan_term, interest_rate, loan_purpose, emi,
                ip_address, geolocation, days_left_to_pay_current_emi, delayed_payment,
                income_level, financial_status
            ) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
        ''', (
            row[0], dob, row[2], row[3], phone_number, row[5], row[6], row[7], row[8],
            int(row[9]), row[10], float(row[11]), int(row[12]), float(row[13]), row[14],
            float(row[15]), row[16], row[17], int(row[19]), row[20], income_level, financial_status
        ))

        # Get the last inserted row's ID
        borrower_id = cursor.lastrowid

        # Parse the repayment history string
        repayment_history_str = row[18]
        repayment_history = eval(repayment_history_str)

        # Insert data into the repayment_history table
        for payment in repayment_history:
            payment_date = payment['Payment Date']
            payment_mode = payment['Payment Mode']
            cursor.execute('''
                INSERT INTO repayment_history (
                    borrower_id, payment_date, payment_mode
                ) VALUES (?,?,?)
            ''', (borrower_id, payment_date, payment_mode))

# Update email addresses in the borrowers table
cursor.execute('''
    UPDATE borrowers
    SET email_address = REPLACE(email_address, SUBSTR(email_address, INSTR(email_address, '@') + 1), 'gmail.com')
''')

# Commit the changes and close the database connection
db_conn.commit()
db_conn.close()

Skipping row with insufficient columns: ['A']


# Visualize the borrower data using Python's matplotlib and seaborn libraries:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

# Connect to the SQLite database
db_conn = sqlite3.connect('debt_collection.db')
cursor = db_conn.cursor()

# Retrieve the borrower data
cursor.execute('SELECT * FROM borrowers')
borrowers = cursor.fetchall()

# Convert the data to a Pandas DataFrame
import pandas as pd
borrowers_df = pd.DataFrame(borrowers, columns=[desc[0] for desc in cursor.description])

# Visualize the data

# Age distribution
plt.figure(figsize=(8, 6))
sns.histplot(borrowers_df['date_of_birth'].dt.year, bins=20)
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()

# Gender distribution
plt.figure(figsize=(8, 6))
sns.countplot(x='gender', data=borrowers_df)
plt.title('Gender Distribution')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.show()

# Marital status distribution
plt.figure(figsize=(8, 6))
sns.countplot(x='marital_status', data=borrowers_df)
plt.title('Marital Status Distribution')
plt.xlabel('Marital Status')
plt.ylabel('Count')
plt.show()

# Language preference distribution
plt.figure(figsize=(8, 6))
sns.countplot(x='language_preference', data=borrowers_df)
plt.title('Language Preference Distribution')
plt.xlabel('Language Preference')
plt.ylabel('Count')
plt.show()

# Loan type distribution
plt.figure(figsize=(8, 6))
sns.countplot(x='loan_type', data=borrowers_df)
plt.title('Loan Type Distribution')
plt.xlabel('Loan Type')
plt.ylabel('Count')
plt.show()

# Credit score distribution
plt.figure(figsize=(8, 6))
sns.histplot(borrowers_df['credit_score'], bins=20)
plt.title('Credit Score Distribution')
plt.xlabel('Credit Score')
plt.ylabel('Count')
plt.show()

# Close the database connection
db_conn.close()

# generate a collection risk score for every borrower and rank them based on this score:

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Load the borrower data
db_conn = sqlite3.connect('debt_collection.db')
cursor = db_conn.cursor()
cursor.execute('SELECT * FROM borrowers')
borrowers = cursor.fetchall()
borrowers_df = pd.DataFrame(borrowers, columns=[desc[0] for desc in cursor.description])

# Define the features and target variable for the collection risk model
features = ['credit_score', 'loan_amount', 'loan_term', 'interest_rate', 'days_left_to_pay_current_emi', 'delayed_payment']
target = 'defaulted'

# Create a logistic regression model to predict the probability of default
X = borrowers_df[features]
y = borrowers_df[target]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
model = LogisticRegression()
model.fit(X_scaled, y)

# Generate the collection risk score for each borrower
collection_risk_scores = model.predict_proba(X_scaled)[:, 1]

# Add the collection risk score to the borrower data
borrowers_df['collection_risk_score'] = collection_risk_scores

# Rank the borrowers based on their collection risk score
borrowers_df['rank'] = borrowers_df['collection_risk_score'].rank(ascending=False)

# Print the top 10 borrowers with the highest collection risk score
print(borrowers_df.sort_values('rank').head(10))

# Close the database connection
db_conn.close()

In [ ]:
import pandas as pd
import numpy as np

# Load the borrower data
db_conn = sqlite3.connect('debt_collection.db')
cursor = db_conn.cursor()
cursor.execute('SELECT * FROM borrowers')
borrowers = cursor.fetchall()
borrowers_df = pd.DataFrame(borrowers, columns=[desc[0] for desc in cursor.description])

# Define the factors and weights for the collection risk score
factors = ['payment_history', 'outstanding_debt_amount', 'income_level', 'employment_status', 'credit_score']
weights = [0.30, 0.20, 0.20, 0.15, 0.15]

# Normalize the factors to a common scale
for factor in factors:
    borrowers_df[factor + '_norm'] = (borrowers_df[factor] - borrowers_df[factor].min()) / (borrowers_df[factor].max() - borrowers_df[factor].min())

# Calculate the collection risk score for each borrower
borrowers_df['collection_risk_score'] = 0
for i, factor in enumerate(factors):
    borrowers_df['collection_risk_score'] += weights[i] * borrowers_df[factor + '_norm']

# Rank the borrowers based on their collection risk score
borrowers_df['rank'] = borrowers_df['collection_risk_score'].rank(ascending=False)

# Print the top 10 borrowers with the highest collection risk score
print(borrowers_df.sort_values('rank').head(10))

# Close the database connection
db_conn.close()